In [14]:
import glob, os
import numpy as np
import IPython
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
fontSize =27
plt.rcParams.update({'font.size': fontSize})
import pickle
import matplotlib.ticker as mticker
from joblib import load, dump

In [15]:
pathSave = "Data_Files/"

In [16]:
class plotHeatMap:
    def __init__(self,  _observables, _parameters, _klArray, _yLabel, _xLabel, _plotString, _pathSave, _title, _stdArray={}, figSize=(18,10), fontSize=fontSize):
        self.parameters = _parameters
        self.observables = _observables
        self.klArray = _klArray
        self.stdArray = _stdArray
        self.yLabel = _yLabel
        self.xLabel = _xLabel
        self.plotString = _plotString
        self.pathSave = _pathSave
        self.title = _title
        self.figSize = figSize
        self.fontSize = fontSize
        
    def heatmap(self, data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
        """
        Create a heatmap from a numpy array and two lists of labels.

        Arguments:
            data       : A 2D numpy array of shape (N,M)
            row_labels : A list or array of length N with the labels
                         for the rows
            col_labels : A list or array of length M with the labels
                         for the columns
        Optional arguments:
            ax         : A matplotlib.axes.Axes instance to which the heatmap
                         is plotted. If not provided, use current axes or
                         create a new one.
            cbar_kw    : A dictionary with arguments to
                         :meth:`matplotlib.Figure.colorbar`.
            cbarlabel  : The label for the colorbar
        All other arguments are directly passed on to the imshow call.
        """
        if not ax:
            ax = plt.gca()

        # Plot the heatmap
        im = ax.imshow(data, **kwargs)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="5%", pad=0.2)
        # Create colorbar
        cbar = ax.figure.colorbar(im, cax=cax, **cbar_kw)
        cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

        # We want to show all ticks...
        ax.set_xticks(np.arange(data.shape[1]))
        ax.set_yticks(np.arange(data.shape[0]))
        # ... and label them with the respective list entries.

        ax.set_xticklabels(col_labels)
        ax.set_yticklabels(row_labels)
    
        # Let the horizontal axes labeling appear on top.
        ax.tick_params(top=True, bottom=False,
                       labeltop=True, labelbottom=False)
        
        ax.set_xlabel(self.xLabel)
        ax.set_ylabel(self.yLabel)
        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=0, ha="right", #=-30
                 rotation_mode="anchor")

        # Turn spines off and create white grid.
        for edge, spine in ax.spines.items():
            spine.set_visible(False)

        ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
        ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
        ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
        ax.tick_params(which="minor", bottom=False, left=False)

        return im, cbar


    def annotate_heatmap(self, im, stdBool=False, data=None, valfmt="{x:.2f}",
                         textcolors=["black", "white"],
                         threshold=None, **textkw):
        """
        A function to annotate a heatmap.

        Arguments:
            im         : The AxesImage to be labeled.
        Optional arguments:
            data       : Data used to annotate. If None, the image's data is used.
            valfmt     : The format of the annotations inside the heatmap.
                         This should either use the string format method, e.g.
                         "$ {x:.2f}", or be a :class:`matplotlib.ticker.Formatter`.
            textcolors : A list or array of two color specifications. The first is
                         used for values below a threshold, the second for those
                         above.
            threshold  : Value in data units according to which the colors from
                         textcolors are applied. If None (the default) uses the
                         middle of the colormap as separation.

        Further arguments are passed on to the created text labels.
        """

        if not isinstance(data, (list, np.ndarray)):
            data = im.get_array()

        # Normalize the threshold to the images color range.
        if threshold is not None:
            threshold = im.norm(threshold)
        else:
            threshold = im.norm(data.max())/2.3

        # Set default alignment to center, but allow it to be
        # overwritten by textkw.
        kw = dict(horizontalalignment="center",
                  verticalalignment="bottom")
        kw.update(textkw)
        
        kw1 = dict(horizontalalignment="center",
                  verticalalignment="top")
        kw1.update(textkw)

        # Get the formatter in case a string is supplied
        if isinstance(valfmt, str):
            valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

        # Loop over the data and create a `Text` for each "pixel".
        # Change the text's color depending on the data.
        texts = []
        for i in range(data.shape[0]):
            for j in range(data.shape[1]):
                if not stdBool:
                    kw.update(color=textcolors[im.norm(data[i, j]) > threshold])
                    text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
                else:
                    kw1.update(color=textcolors[im.norm(data[i, j]) > threshold])
                    text = im.axes.text(j, i, r'$\pm$' + valfmt(self.stdArray[i,j], None), **kw1)
                texts.append(text)

        return texts
    
    def plotMap(self):
        plt.rcParams.update({'font.size': self.fontSize})
        fig, ax = plt.subplots(figsize=self.figSize,dpi=480)
        im, cbar = self.heatmap(self.klArray,  self.parameters, self.observables, ax=ax, cmap="YlGn", cbarlabel=self.title) #"$KL(MP||Ref)-KL(NN||Ref)$")
        texts = self.annotate_heatmap(im, False, valfmt="{x:.2f}", size=self.fontSize/1.1)
        texts = self.annotate_heatmap(im, True, valfmt="{x:.2f}", size=self.fontSize/1.5)
        fig.tight_layout()
        #fig.savefig(self.pathSave + "/Pics/" + self.plotString + '.pdf', bbox_inches='tight')
        plt.show()


In [ ]:
with open(pathSave + "Parameters_Observables.txt", "rb") as fkl:
    Dict_Parameters_Observables = load(fkl)

klArray = Dict_Parameters_Observables['log-likelihood_mean']
stdArray = Dict_Parameters_Observables['log-likelihood_std']
x = Dict_Parameters_Observables['x']
y = Dict_Parameters_Observables['y']
xLabel = Dict_Parameters_Observables['xLabel'] 
yLabel = Dict_Parameters_Observables['yLabel']

counter = 0
for rInd in [[0,43],[43,85],[85,127]]:
    _y = y[rInd[0]:rInd[1]]
    _klArray = klArray[rInd[0]:rInd[1],:]
    _klArray.shape = (rInd[1]-rInd[0],5)

    plotString = "Parameters_Observables_" + str(counter)
    HM = plotHeatMap(x, _y, _klArray, yLabel, xLabel, plotString, pathSave \
                     ,"Log likelihood " + r'($\mu \pm \sigma$)', stdArray, (24,24), 15)
    HM.plotMap()
    counter += 1

In [ ]:
with open(pathSave + "Parameters_Observables_Select.txt", "rb") as fkl:
    Dict_Parameters_Observables = load(fkl)

klArray = Dict_Parameters_Observables['log-likelihood_mean']
stdArray = Dict_Parameters_Observables['log-likelihood_std']
x = Dict_Parameters_Observables['x']
y = Dict_Parameters_Observables['y']
xLabel = Dict_Parameters_Observables['xLabel'] 
yLabel = Dict_Parameters_Observables['yLabel']

counter = 0
for rInd in [[0,len(y)]]:
    _y = y[rInd[0]:rInd[1]]
    _klArray = klArray[rInd[0]:rInd[1],:]
    _klArray.shape = (rInd[1]-rInd[0],5)

    plotString = "Parameters_Observables_Select"
    HM = plotHeatMap(x, _y, _klArray, yLabel, xLabel, plotString, pathSave \
                     ,"Log likelihood " + r'($\mu \pm \sigma$)', stdArray, (24,24), 35)
    HM.plotMap()
    counter += 1

In [ ]:
plotString = "Parameters_Observables_Tprof"
with open(pathSave + plotString + ".txt", "rb") as fkl:
    Dict_Parameters_Observables = load(fkl)

klArray = Dict_Parameters_Observables['log-likelihood_mean']
stdArray = Dict_Parameters_Observables['log-likelihood_std']
x = Dict_Parameters_Observables['x']
y = Dict_Parameters_Observables['y']
xLabel = Dict_Parameters_Observables['xLabel'] 
yLabel = Dict_Parameters_Observables['yLabel']

HM = plotHeatMap(x, y, klArray, yLabel, xLabel, plotString, pathSave \
                 ,"Log likelihood " + r'($\mu \pm \sigma$)', stdArray)
HM.plotMap()

In [ ]:
plotString = "Parameters_Observables_Tprof_inverse"
with open(pathSave + plotString + ".txt", "rb") as fkl:
    Dict_Parameters_Observables = load(fkl)

klArray = Dict_Parameters_Observables['log-likelihood_mean']
stdArray = Dict_Parameters_Observables['log-likelihood_std']
x = Dict_Parameters_Observables['x']
y = Dict_Parameters_Observables['y']
xLabel = Dict_Parameters_Observables['xLabel'] 
yLabel = Dict_Parameters_Observables['yLabel']

HM = plotHeatMap(x, y, klArray, yLabel, xLabel, plotString, pathSave \
                 ,"Log likelihood " + r'($\mu \pm \sigma$)', stdArray)
HM.plotMap()

In [ ]:
plotString = "Parameters_Noise"
with open(pathSave + plotString + ".txt", "rb") as fkl:
    Dict_Parameters_Observables = load(fkl)

klArray = Dict_Parameters_Observables['log-likelihood_mean']
stdArray = Dict_Parameters_Observables['log-likelihood_std']
x = Dict_Parameters_Observables['x']
y = Dict_Parameters_Observables['y']
xLabel = Dict_Parameters_Observables['xLabel'] 
yLabel = Dict_Parameters_Observables['yLabel']

HM = plotHeatMap(x, y, klArray, yLabel, xLabel, plotString, pathSave \
                 ,"Log likelihood " + r'($\mu \pm \sigma$)', stdArray)
HM.plotMap()

In [ ]:
for p in [0,4,5,6,7]:
    plotString = "Simulations_Observables" + str(p)
    with open(pathSave + plotString + ".txt", "rb") as fkl:
        Dict_Parameters_Observables = load(fkl)

    klArray = Dict_Parameters_Observables['log-likelihood_mean']
    stdArray = Dict_Parameters_Observables['log-likelihood_std']
    x = Dict_Parameters_Observables['x']
    y = Dict_Parameters_Observables['y']
    xLabel = Dict_Parameters_Observables['xLabel'] 
    yLabel = Dict_Parameters_Observables['yLabel']

    HM = plotHeatMap(x, y, klArray, yLabel, "Number of simulations", plotString, pathSave \
                     ,"Log likelihood " + r'($\mu \pm \sigma$)', stdArray)
    HM.plotMap()